## Runtime Arguments 바인딩

In [1]:
# !pip install -qU langchain langchain-openai

- runnable.invoke() 메서드를 호출하여 "x raised to the third plus seven equals 12"라는 방정식 문장을 전달하고 결과를 출력합니다.

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # 대수 기호를 사용하여 다음 방정식을 작성한 다음 풀이하세요.
            "Write out the following equation using algebraic symbols then solve it. "
            "Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        (
            "human",
            "{equation_statement}",  # 사용자가 입력한 방정식 문장을 변수로 받습니다.
        ),
    ]
)
# ChatOpenAI 모델을 초기화하고 temperature를 0으로 설정합니다.
model = ChatOpenAI(model="gpt-4", temperature=0)

# 방정식 문장을 입력받아 프롬프트에 전달하고, 모델에서 생성된 결과를 문자열로 파싱합니다.
runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

# 예시 방정식 문장을 입력하여 결과를 출력합니다.
print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12
SOLUTION: x^3 = 12 - 7
x^3 = 5
x = ∛5


In [5]:
runnable = (
    # 실행 가능한 패스스루 객체를 생성하여 "equation_statement" 키에 할당합니다.
    {"equation_statement": RunnablePassthrough()}
    | prompt  # 프롬프트를 파이프라인에 추가합니다.
    | model.bind(
        stop="SOLUTION"
    )  # 모델을 바인딩하고 "SOLUTION" 토큰에서 생성을 중지하도록 설정합니다.
    | StrOutputParser()  # 문자열 출력 파서를 파이프라인에 추가합니다.
)
# "x raised to the third plus seven equals 12"라는 입력으로 파이프라인을 실행하고 결과를 출력합니다.
print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x^3 + 7 = 12



### OpenAI Functions 기능 연결

In [6]:
openai_function = {
    "name": "solver",  # 함수의 이름
    # 함수의 설명: 방정식을 수립하고 해결합니다.
    "description": "Formulates and solves an equation",
    "parameters": {  # 함수의 매개변수
        "type": "object",  # 매개변수의 타입: 객체
        "properties": {  # 매개변수의 속성
            "equation": {  # 방정식 속성
                "type": "string",  # 방정식의 타입: 문자열
                "description": "The algebraic expression of the equation",  # 방정식의 대수식 표현
            },
            "solution": {  # 해답 속성
                "type": "string",  # 해답의 타입: 문자열
                "description": "The solution to the equation",  # 방정식의 해답
            },
        },
        "required": ["equation", "solution"],  # 필수 매개변수: 방정식과 해답
    },
}

In [7]:
# 다음 방정식을 대수 기호를 사용하여 작성한 다음 해결하세요.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it.",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(model="gpt-4", temperature=0).bind(
    function_call={"name": "solver"},  # openai_function schema 를 바인딩합니다.
    functions=[openai_function],
)
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
# x의 세제곱에 7을 더하면 12와 같다
runnable.invoke("x raised to the third plus seven equals 12")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n"equation": "x^3 + 7 = 12",\n"solution": "x = ∛5"\n}', 'name': 'solver'}}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 95, 'total_tokens': 122}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-29c1a995-b64f-4db1-929a-9d1e49a6c29b-0', usage_metadata={'input_tokens': 95, 'output_tokens': 27, 'total_tokens': 122})

### OpenAI tools 연결하기

In [8]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",  # 현재 날씨를 가져오는 함수의 이름
            "description": "주어진 위치의 현재 날씨를 가져옵니다",  # 함수에 대한 설명
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "도시와 주, 예: San Francisco, CA",  # 위치 매개변수에 대한 설명
                    },
                    # 온도 단위 매개변수 (섭씨 또는 화씨)
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],  # 필수 매개변수 지정
            },
        },
    }
]

- bind() 메서드를 사용하여 tools를 모델에 바인딩합니다.
- invoke() 메서드를 호출하여 "샌프란시스코, 뉴욕, 로스앤젤레스의 현재 날씨에 대해 알려줘?"라는 질문을 모델에 전달합니다.

In [9]:
# ChatOpenAI 모델을 초기화하고 도구를 바인딩합니다.
model = ChatOpenAI(model="gpt-3.5-turbo").bind(tools=tools)
# 모델을 호출하여 샌프란시스코, 뉴욕, 로스앤젤레스의 날씨에 대해 질문합니다.
model.invoke("샌프란시스코, 뉴욕, 로스앤젤레스의 현재 날씨에 대해 알려줘?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ydx8oXYmPk6zxLwkL6S0CqeC', 'function': {'arguments': '{"location": "San Francisco, CA", "unit": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_EXInQIuJARJHUMmaVCgqmW5y', 'function': {'arguments': '{"location": "New York, NY", "unit": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'call_Kt1wLmsFvnRiZiN5C4iLdYxJ', 'function': {'arguments': '{"location": "Los Angeles, CA", "unit": "celsius"}', 'name': 'get_current_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 127, 'total_tokens': 211}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c0ce152c-9d95-427e-af88-b14f63698725-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'San Francisco, CA', 'unit': 'celsius'}, 'id': 'call_ydx8oXYmPk6zxLwkL6S0CqeC', 'type': 'too